# How to deploy an elasticsearch instance

## Preparation

- Ask your supervisor to create a config file for your account (e.g., `your_student_id-k8s-config`)

## Steps

:bulb: Run the following commands from the terminal in the Jupyter Notebook

- Prepare for `kubectl`

```
$ mkdir ~/.kube
$ cp ~/shared/Datasets/k8s/certs/your_student_id-k8s-config ~/.kube/config
```

- Check the config

```
$ kubectl get all
No resources ...
```

- Prepare for your project folder (e.g., `my-eck-project`)

```
$ mkdir ~/playground/projects/your_name/my-eck-project
$ cd ~/playground/projects/your_name/my-eck-project

```

- Create `eck-1.yaml`
  - Node: 1
  - HDD: 1Gi

```
apiVersion: elasticsearch.k8s.elastic.co/v1
kind: Elasticsearch
metadata:
  name: my-eck
spec:
  version: 7.9.0
  nodeSets:
  - name: default
    count: 1
    config:
      node.master: true
      node.data: true
      node.ingest: true
      node.store.allow_mmap: false
```

- and apply

```
$ kubectl apply -f eck-1.yaml
```

- Wait until the server health status turns green

```
$ kubectl get elasticsearch
NAME     HEALTH   NODES   VERSION   PHASE   AGE
my-eck   green    1       7.9.0     Ready   97s
```

- Expose the instance to the cluster nodes
  - Choose a port number bewteen 30000-32767
  - Try another one if you get an error

```
$ kubectl patch service my-eck-es-http --type='json' -p='[{"op": "replace", "path": "/spec/type", "value": "NodePort"}]'
$ kubectl patch service my-eck-es-http --type='json' -p='[{"op": "replace", "path": "/spec/ports/0/nodePort", "value": YOUR_PORT_NUMBER}]'
```

- Get a password

```
$ kubectl get secret my-eck-es-elastic-user -o=jsonpath='{.data.elastic}' | base64 --decode; echo
```

- Run this notebook
---

In [ ]:
import sys
!{sys.executable} -m pip install elasticsearch

In [ ]:
from datetime import datetime
from elasticsearch import Elasticsearch
USER = 'elastic'
PASS = 'YOUR_PASSWORD'
HOST = 'isr1'
PORT = 'YOUR_PORT_NUMBER'

es = Elasticsearch(
    ['https://' + USER + ':' + PASS + '@' + HOST + ':' + PORT],
    # no verify SSL certificates
    verify_certs=False,
    # don't show warnings about ssl certs verification
    ssl_show_warn=False
)

In [ ]:
doc = {
    'author': 'kimchy',
    'text': 'Elasticsearch: cool. bonsai cool.',
    'timestamp': datetime.now(),
}
res = es.index(index="test-index", id=1, body=doc)
print(res['result'])

In [ ]:
res = es.get(index="test-index", id=1)
print(res['_source'])

In [ ]:
es.indices.refresh(index="test-index")

res = es.search(index="test-index", body={"query": {"match_all": {}}})
print("Got %d Hits:" % res['hits']['total']['value'])
for hit in res['hits']['hits']:
    print("%(timestamp)s %(author)s: %(text)s" % hit["_source"])

---
## How to expand the spec of Elasticsearch

- Learn how to use Elasticsearch with the above setting (i.e., `eck-1.yaml`)
- When you're ready to try a bigger dataset, run the following commands

### Steps

- Stop the existing instance

```
$ kubectl delete -f eck-1.yaml
elasticsearch.elasticsearch.k8s.elastic.co "my-eck" deleted
```

- Check if the instance has stopped

```
$ kubectl get elastisearch
error: the server doesn't have a resource type "elastisearch"
```

- Create `eck-3.yaml`
  - Node: 3
  - HDD: 100GB max

```
apiVersion: elasticsearch.k8s.elastic.co/v1
kind: Elasticsearch
metadata:
  name: my-eck
spec:
  version: 7.9.0
  nodeSets:
  - name: default
    count: 3
    config:
      node.master: true
      node.data: true
      node.ingest: true
      node.store.allow_mmap: false
    volumeClaimTemplates:
    - metadata:
        name: elasticsearch-data
      spec:
        accessModes:
        - ReadWriteOnce
        resources:
          requests:
            storage: 100Gi
        storageClassName: nfs
```

- and apply

```
kubectl apply -f eck-3.yaml
```

- Wait until the server health status turns green

```
$ kubectl get elasticsearch
NAME     HEALTH   NODES   VERSION   PHASE   AGE
my-eck   green    3       7.9.0     Ready   97s
```

- Expose the instance to the cluster nodes
  - Choose a port number bewteen 30000-32767
  - Try another one if you get an error

```
$ kubectl patch service my-eck-es-http --type='json' -p='[{"op": "replace", "path": "/spec/type", "value": "NodePort"}]'
$ kubectl patch service my-eck-es-http --type='json' -p='[{"op": "replace", "path": "/spec/ports/0/nodePort", "value": YOUR_PORT_NUMBER}]'
```

- Get a password

```
$ PASSWORD=$(kubectl get secret my-eck-es-elastic-user -o go-template='{{.data.elastic | base64decode}}')
$ echo $PASSWORD
```

- Run this notebook


### Delete the instance

:warning: This will delete everything including index files. Make sure to backup important files/data before doing this.

```
$ kubectl delete -f eck-3.yaml
```